In [ ]:
#!pip install selenium

In [3]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException # 예외사항
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from tqdm import tqdm as tq  # 진행상황 시각화
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import requests as req
import time
import os   # 폴더, 파일생성 라이브러리
import pickle # 리스트 저장, 불러오기 위함
import random as rd

In [5]:
driver = wb.Chrome()
driver.get("https://www.musinsa.com/search/musinsa/goods?q=%EB%82%A8%EC%9E%90+%EC%83%81%EC%9D%98&category1=001%3A%EC%83%81%EC%9D%98%3Atrue%3A")
time.sleep(0.5)
driver.find_element(By.CSS_SELECTOR, "#root > div.sc-1fn96ju-0.exZiif > main > div > div.sc-164fymo-1.dCSxxu > button").click()

In [ ]:
cate_list = ['half_t_shirt','shirt','sports_top','no_arm','collar_t_shirt','long_arm','hood_t_shirt','man_2_man']

In [35]:
# 의류 크롤링

color_link = ['2%3A%EA%B2%80%EC%A0%95%EC%83%89%3Atrue%3A', '1%3A%ED%9D%B0%EC%83%89%3Atrue%3A','36%3A%EB%84%A4%EC%9D%B4%EB%B9%84%3Atrue%3A','24%3A%EB%9D%BC%EAC%9D%B4%ED%8A%B8+%EA%B7%B8%EB%A0%88%EC%9D%B4%3Atrue%3','25%3A%EB%8B%A4%ED%81%AC+%EA%B7%B8%EB%A0%88%EC%9D%B4%3Atrue%3A','3%3A%ED%9A%8C%EC%83%89%3Atrue%3A','7%3A%ED%8C%8C%EB%9E%80%EC%83%89%3Atrue%3A','23%3A%EC%95%84%EC%9D%B4%EB%B3%B4%EB%A6%AC%3Atrue%3A','37%3A%EC%8A%A4%EC%B9%B4%EC%9D%B4+%EB%B8%94%EB%A3%A8%3Atrue%3A','5%3A%EB%B2%A0%EC%9D%B4%EC%A7%80%EC%83%89%3Atrue%3A','6%3A%EB%85%B9%EC%83%89%3Atrue%3A']

color_list = ['black','white','navy','lightgray','darkgray','gray','blue','ivory','skyblue','beige','green']

cate_link = ['001001%3A%EB%B0%98%EC%86%8C%EB%A7%A4+%ED%8B%B0%EC%85%94%EC%B8%A0%3Atrue%3A','001002%3A%EC%85%94%EC%B8%A0%2F%EB%B8%94%EB%9D%BC%EC%9A%B0%EC%8A%A4%3Atrue%3A&','001013%3A%EC%8A%A4%ED%8F%AC%EC%B8%A0+%EC%83%81%EC%9D%98%3Atrue%3A&','001011%3A%EB%AF%BC%EC%86%8C%EB%A7%A4+%ED%8B%B0%EC%85%94%EC%B8%A0%3Atrue%3A','001003%3A%ED%94%BC%EC%BC%80%2F%EC%B9%B4%EB%9D%BC+%ED%8B%B0%EC%85%94%EC%B8%A0%3Atrue%3A','001010%3A%EA%B8%B4%EC%86%8C%EB%A7%A4+%ED%8B%B0%EC%85%94%EC%B8%A0%3Atrue%3A','001004%3A%ED%9B%84%EB%93%9C+%ED%8B%B0%EC%85%94%EC%B8%A0%3Atrue%3A','001005%3A%EB%A7%A8%ED%88%AC%EB%A7%A8%2F+%EC%8A%A4%EC%9B%A8%ED%8A%B8%EC%85%94%EC%B8%A0%3Atrue%3A']

cate_list = ['half_t_shirt','shirt','sports_top','no_arm','collar_t_shirt','long_arm','hood_t_shirt','man_2_man']

DF_cloth_half_t_shirt = pd.DataFrame(columns=['name', 'cate', 'imgurl', 'url', 'cate_detail', 'color', 'price', 'dc_price'])
DF_clothone = pd.DataFrame({})
DF_size_half_t_shirt = pd.DataFrame(columns=['name', 'size', 'top_length', 'sh_length', 'ch_length', 'hn_length'])
DF_sizeone = pd.DataFrame({})

cate_num = 2

for col in tq(range(len(color_link))):
    current_url = f"https://www.musinsa.com/search/musinsa/goods?q=%EB%82%A8%EC%9E%90+%EC%83%81%EC%9D%98&category1=001%3A%EC%83%81%EC%9D%98%3Atrue%3A&category2={cate_link[cate_num]}&color={color_link[col]}"
    print(f"{col} 크롤링")
    driver.get(current_url)
    for i in range(1,41):
        try:
            ran = (rd.randint(10, 25)) / 10
            time.sleep(0.3)
            name = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-9.ftECEJ > a").text
            cate = 'top'
            cate_detail = cate_list[cate_num]
            color = color_list[col]
            imgurl = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-5.bOgxyc > figure > img").get_attribute('src')
            url = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-5.bOgxyc > figure > a").get_attribute('href')
            price = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-9.ftECEJ > div.sc-1dubb4w-0.MfYCC > div > div > button > strong").text
            try:
                dc_price = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-9.ftECEJ > div.sc-1dubb4w-0.MfYCC > div > div > del").text
            except NoSuchElementException:
                dc_price = 0
                print(f"할인가격 없음 : {i}")    
            time.sleep(ran)
                            
            try:
                driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-5.bOgxyc > figure > a").click()
            except ElementClickInterceptedException:
                print(f"url 주소 불일치 : {i}")
                continue

            time.sleep(ran / 2)
            actions = driver.find_element(By.CSS_SELECTOR, 'body')
            actions.send_keys(Keys.END)
            time.sleep(ran / 2)

            sizeLength = len(driver.find_elements(By.CSS_SELECTOR, "#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > ul > li"))
            if sizeLength > 0:
                if len(driver.find_elements(By.CSS_SELECTOR, "#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > div > table > tbody > tr:nth-child(2) > td")) != 4:
                    print(f"데이터 수 불일치 : {i}")
                    driver.back()
                    continue
                else:
                    for j in range(3, sizeLength + 1):
                        size = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > ul > li:nth-child({j})").text
                        top_length = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > div > table > tbody > tr:nth-child({j - 1}) > td:nth-child(1)").text
                        sh_length = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > div > table > tbody > tr:nth-child({j - 1}) > td:nth-child(2)").text
                        ch_length = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > div > table > tbody > tr:nth-child({j - 1}) > td:nth-child(3)").text
                        hn_length = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > div > table > tbody > tr:nth-child({j - 1}) > td:nth-child(4)").text

                        DF_sizeone = pd.DataFrame({'name': [name], 'size': [size], 'top_length': [top_length], 'sh_length': [sh_length], 'ch_length': [ch_length], 'hn_length': [hn_length]})
                        DF_size_half_t_shirt = pd.concat([DF_size_half_t_shirt, DF_sizeone], axis=0)

                DF_clothone = pd.DataFrame({'name': [name], 'cate': [cate], 'imgurl': [imgurl], 'url': [url], 'cate_detail': [cate_detail], 'color': [color], 'price': [price], 'dc_price': [dc_price]})
                DF_cloth_half_t_shirt = pd.concat([DF_cloth_half_t_shirt, DF_clothone], axis=0)
                print(f"성공 : {i}")
                driver.back()
            else:
                print(f"사이즈 정보 없음 : {i}")
                driver.back()
        except NoSuchElementException:
            print(f"오류 발생 : {i}")
            driver.get(current_url)
            continue

DF_cloth_half_t_shirt['index'] = range(len(DF_cloth_half_t_shirt))
DF_cloth_half_t_shirt['index'] += 1
DF_cloth_half_t_shirt = DF_cloth_half_t_shirt.set_index('index')

DF_size_half_t_shirt['index'] = range(len(DF_size_half_t_shirt))
DF_size_half_t_shirt['index'] += 1
DF_size_half_t_shirt = DF_size_half_t_shirt.set_index('index')

DF_cloth_half_t_shirt.to_csv('DF_cloth_half_t_shirt', encoding='utf-8')
DF_size_half_t_shirt.to_csv('DF_size_half_t_shirt', encoding='utf-8')

  0%|          | 0/11 [00:00<?, ?it/s]

0 크롤링
사이즈 정보 없음 : 1
url 주소 불일치 : 2
할인가격 없음 : 3
성공 : 3
데이터 수 불일치 : 4
성공 : 5
성공 : 6
사이즈 정보 없음 : 7
데이터 수 불일치 : 8
성공 : 9
할인가격 없음 : 10
사이즈 정보 없음 : 10
할인가격 없음 : 11
사이즈 정보 없음 : 11
성공 : 12
성공 : 13
할인가격 없음 : 14
성공 : 14
할인가격 없음 : 15
데이터 수 불일치 : 15
성공 : 16
성공 : 17
사이즈 정보 없음 : 18
성공 : 19
사이즈 정보 없음 : 20
성공 : 21
할인가격 없음 : 22
성공 : 22
할인가격 없음 : 23
사이즈 정보 없음 : 23
성공 : 24
할인가격 없음 : 25
성공 : 25
오류 발생 : 26
오류 발생 : 27
오류 발생 : 28
오류 발생 : 29
오류 발생 : 30
오류 발생 : 31
오류 발생 : 32
오류 발생 : 33
오류 발생 : 34
오류 발생 : 35
오류 발생 : 36
오류 발생 : 37
오류 발생 : 38
오류 발생 : 39
오류 발생 : 40


  9%|▉         | 1/11 [03:18<33:03, 198.37s/it]

1 크롤링
사이즈 정보 없음 : 1
url 주소 불일치 : 2
사이즈 정보 없음 : 3
사이즈 정보 없음 : 4
할인가격 없음 : 5
사이즈 정보 없음 : 5
성공 : 6
성공 : 7
할인가격 없음 : 8
성공 : 8
할인가격 없음 : 9
사이즈 정보 없음 : 9
성공 : 10
사이즈 정보 없음 : 11
성공 : 12
성공 : 13
성공 : 14
할인가격 없음 : 15
사이즈 정보 없음 : 15
사이즈 정보 없음 : 16
사이즈 정보 없음 : 17
할인가격 없음 : 18
데이터 수 불일치 : 18
사이즈 정보 없음 : 19
할인가격 없음 : 20
데이터 수 불일치 : 20
사이즈 정보 없음 : 21
사이즈 정보 없음 : 22
성공 : 23
성공 : 24
사이즈 정보 없음 : 25
오류 발생 : 26
오류 발생 : 27
오류 발생 : 28
오류 발생 : 29
오류 발생 : 30
오류 발생 : 31
오류 발생 : 32
오류 발생 : 33
오류 발생 : 34
오류 발생 : 35
오류 발생 : 36
오류 발생 : 37
오류 발생 : 38
오류 발생 : 39
오류 발생 : 40


 18%|█▊        | 2/11 [06:22<28:28, 189.78s/it]

...

 82%|████████▏ | 9/11 [27:49<06:06, 183.30s/it]

9 크롤링
사이즈 정보 없음 : 1
url 주소 불일치 : 2
사이즈 정보 없음 : 3
사이즈 정보 없음 : 4
할인가격 없음 : 5
데이터 수 불일치 : 5
할인가격 없음 : 6
성공 : 6
할인가격 없음 : 7
성공 : 7
사이즈 정보 없음 : 8
할인가격 없음 : 9
성공 : 9
성공 : 10
성공 : 11
성공 : 12
할인가격 없음 : 13
데이터 수 불일치 : 13
사이즈 정보 없음 : 14
사이즈 정보 없음 : 15
할인가격 없음 : 16
성공 : 16
사이즈 정보 없음 : 17
성공 : 18
사이즈 정보 없음 : 19
할인가격 없음 : 20
사이즈 정보 없음 : 20
할인가격 없음 : 21
사이즈 정보 없음 : 21
할인가격 없음 : 22
사이즈 정보 없음 : 22
할인가격 없음 : 23
사이즈 정보 없음 : 23
할인가격 없음 : 24
사이즈 정보 없음 : 24
할인가격 없음 : 25
사이즈 정보 없음 : 25
오류 발생 : 26
오류 발생 : 27
오류 발생 : 28
오류 발생 : 29
오류 발생 : 30
오류 발생 : 31
오류 발생 : 32
오류 발생 : 33
오류 발생 : 34
오류 발생 : 35
오류 발생 : 36
오류 발생 : 37
오류 발생 : 38
오류 발생 : 39
오류 발생 : 40


 91%|█████████ | 10/11 [30:51<03:02, 183.00s/it]

10 크롤링
성공 : 1
할인가격 없음 : 2
url 주소 불일치 : 2
성공 : 3
성공 : 4
성공 : 5
할인가격 없음 : 6
성공 : 6
할인가격 없음 : 7
사이즈 정보 없음 : 7
성공 : 8
데이터 수 불일치 : 9
할인가격 없음 : 10
데이터 수 불일치 : 10
사이즈 정보 없음 : 11
할인가격 없음 : 12
사이즈 정보 없음 : 12
할인가격 없음 : 13
사이즈 정보 없음 : 13
할인가격 없음 : 14
사이즈 정보 없음 : 14
사이즈 정보 없음 : 15
사이즈 정보 없음 : 16
성공 : 17
사이즈 정보 없음 : 18
데이터 수 불일치 : 19
사이즈 정보 없음 : 20
데이터 수 불일치 : 21
할인가격 없음 : 22
성공 : 22
할인가격 없음 : 23
성공 : 23
할인가격 없음 : 24
데이터 수 불일치 : 24
사이즈 정보 없음 : 25
오류 발생 : 26
오류 발생 : 27
오류 발생 : 28
오류 발생 : 29
오류 발생 : 30
오류 발생 : 31
오류 발생 : 32
오류 발생 : 33
오류 발생 : 34
오류 발생 : 35
오류 발생 : 36
오류 발생 : 37
오류 발생 : 38
오류 발생 : 39
오류 발생 : 40


100%|██████████| 11/11 [33:28<00:00, 182.62s/it]


In [ ]:
cate_list = ['sport_pants','half_pants','training_pants','slacks_pants','denim_pants','cotton_pants']

In [6]:
# 바지 크롤링
color_link = ['2%3A검정색%3Atrue%3A', '1%3A흰색%3Atrue%3A','36%3A네이비%3Atrue%3A','24%3A라이트+그레이%3Atrue%3A','25%3A다크+그레이%3Atrue%3A','3%3A회색%3Atrue%3A','7%3A파란색%3Atrue%3A','23%3A아이보리%3Atrue%3A','37%3A스카이+블루%3Atrue%3A','5%3A베이지색%3Atrue%3A','6%3A녹색%3Atrue%3A']

color_list = ['black','white','navy','lightgray','darkgray','gray','blue','ivory','skyblue','beige','green']

# color_link = ['2%3A검정색%3Atrue%3A', '1%3A흰색%3Atrue%3A','36%3A네이비%3Atrue%3A','24%3A라이트+그레이%3Atrue%3A','25%3A다크+그레이%3Atrue%3A','3%3A회색%3Atrue%3A','7%3A파란색%3Atrue%3A','23%3A아이보리%3Atrue%3A','37%3A스카이+블루%3Atrue%3A','5%3A베이지색%3Atrue%3A','6%3A녹색%3Atrue%3A','16%3A데님%3Atrue%3A','57%3A연청%3Atrue%3A','58%3A중청%3Atrue%3A','59%3A진청%3Atrue%3A','60%3A흑청%3Atrue%3A']
# color_list = ['black','white','navy','lightgray','darkgray','gray','blue','ivory','skyblue','beige','green', 'denim','lightblue','midblue','deepblue','darkblue']

cate_link = ['003011%3A스포츠+하의%3Atrue%3A','003009%3A숏+팬츠%3Atrue%3A','003004%3A트레이닝%2F조거+팬츠%3Atrue%3A','003008%3A슈트+팬츠%2F슬랙스%3Atrue%3A','003002%3A데님+팬츠%3Atrue%3A','003007%3A코튼+팬츠%3Atrue%3A']

cate_list = ['sport_pants','half_pants','training_pants','slacks_pants','denim_pants','cotton_pants']

DF_cloth_half_pants = pd.DataFrame(columns=['name', 'cate', 'imgurl', 'url', 'cate_detail', 'color', 'price', 'dc_price'])
DF_clothone = pd.DataFrame({})
DF_size_half_pants = pd.DataFrame(columns=['name', 'size', 'waist', 'thigh', 'bot_length', 'hem'])
DF_sizeone = pd.DataFrame({})

cate_num = 1

for col in tq(range(len(color_link))):
    current_url = f"https://www.musinsa.com/search/musinsa/goods?q=%EB%82%A8%EC%9E%90+%ED%95%98%EC%9D%98&category1=003%3A%EB%B0%94%EC%A7%80%3Atrue%3A&category2={cate_link[cate_num]}&color={color_link[col]}"
    print(f"{col} 크롤링")
    driver.get(current_url)
    time.sleep(0.5)
    for i in range(1,41):
        try:
            ran = (rd.randint(10, 25)) / 10
            time.sleep(0.3)
            name = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-9.ftECEJ > a").text
            cate = 'b'
            cate_detail = cate_list[cate_num]
            color = color_list[col]
            imgurl = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-5.bOgxyc > figure > img").get_attribute('src')
            url = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-5.bOgxyc > figure > a").get_attribute('href')
            price = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-9.ftECEJ > div.sc-1dubb4w-0.MfYCC > div > div > button > strong").text
            try:
                dc_price = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-9.ftECEJ > div.sc-1dubb4w-0.MfYCC > div > div > del").text
            except NoSuchElementException:
                dc_price = 0
                print(f"할인가격 없음 : {i}")    
            time.sleep(ran)
                            
            try:
                driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-1fn96ju-0.exZiif > main > section > div > div:nth-child(1) > div > div:nth-child({i}) > div > div.sc-1yenj15-5.bOgxyc > figure > a").click()
            except ElementClickInterceptedException:
                print(f"url 주소 불일치 : {i}")
                continue

            time.sleep(ran / 2)
            actions = driver.find_element(By.CSS_SELECTOR, 'body')
            actions.send_keys(Keys.END)
            time.sleep(ran / 2)

            sizeLength = len(driver.find_elements(By.CSS_SELECTOR, "#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > ul > li"))
            if sizeLength > 0:
                if len(driver.find_elements(By.CSS_SELECTOR, "#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > div > table > tbody > tr:nth-child(2) > td")) >= 4:
                    for j in range(3, sizeLength + 1):
                        
                        tag_size = "#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > div > table > thead > tr > th"
                        
                        size = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > ul > li:nth-child({j})").text
                        for tag in range(len(driver.find_elements(By.CSS_SELECTOR, tag_size))):
                            if(driver.find_elements(By.CSS_SELECTOR, f"{tag_size}:nth-child({tag+1})")[0].text[2:] == '총장'):
                                bot_length = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > div > table > tbody > tr:nth-child({j - 1}) > td:nth-child({tag+1})").text
                            elif(driver.find_elements(By.CSS_SELECTOR, f"{tag_size}:nth-child({tag+1})")[0].text[2:] == '허리단면'):
                                waist = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > div > table > tbody > tr:nth-child({j - 1}) > td:nth-child({tag+1})").text
                            elif(driver.find_elements(By.CSS_SELECTOR, f"{tag_size}:nth-child({tag+1})")[0].text[2:] == '허벅지단면'):
                                thigh = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > div > table > tbody > tr:nth-child({j - 1}) > td:nth-child({tag+1})").text
                            elif(driver.find_elements(By.CSS_SELECTOR, f"{tag_size}:nth-child({tag+1})")[0].text[2:] == '밑단단면'):
                                hem = driver.find_element(By.CSS_SELECTOR, f"#root > div.sc-8beamu-0.hOombb > div.sc-of3kep-0.edZdA-D > div.sc-1sxlp32-0.ijnrws > div > table > tbody > tr:nth-child({j - 1}) > td:nth-child({tag+1})").text
                                
                        DF_sizeone = pd.DataFrame({'name': [name], 'size': [size], 'bot_length': [bot_length], 'waist': [waist], 'thigh': [thigh], 'hem': [hem]})
                        DF_size_half_pants = pd.concat([DF_size_half_pants, DF_sizeone], axis=0)
                else:
                    print(f"데이터 수 불일치 : {i}")
                    driver.back()
                    continue

                DF_clothone = pd.DataFrame({'name': [name], 'cate': [cate], 'imgurl': [imgurl], 'url': [url], 'cate_detail': [cate_detail], 'color': [color], 'price': [price], 'dc_price': [dc_price]})
                DF_cloth_half_pants = pd.concat([DF_cloth_half_pants, DF_clothone], axis=0)
                print(f"성공 : {i}")
                driver.back()
            else:
                print(f"사이즈 정보 없음 : {i}")
                driver.back()
        except NoSuchElementException:
            print(f"오류 발생 : {i}")
            driver.get(current_url)
            continue

DF_cloth_half_pants['index'] = range(len(DF_cloth_half_pants))
DF_cloth_half_pants['index'] += 1
DF_cloth_half_pants = DF_cloth_half_pants.set_index('index')

DF_size_half_pants['index'] = range(len(DF_size_half_pants))
DF_size_half_pants['index'] += 1
DF_size_half_pants = DF_size_half_pants.set_index('index')

DF_cloth_half_pants.iloc[:,7]
for i in range(len(DF_cloth_half_pants)):
    if(DF_cloth_half_pants.iloc[i,7] == '0'):
        pass
    else:
        keep = DF_cloth_half_pants.iloc[i,7]
        DF_cloth_half_pants.iloc[i,7] = DF_cloth_half_pants.iloc[i,6]
        DF_cloth_half_pants.iloc[i,6] = keep

DF_cloth_half_pants.to_csv('DF_cloth_half_pants', encoding='utf-8')
DF_size_half_pants.to_csv('DF_size_half_pants', encoding='utf-8')

  0%|          | 0/11 [00:00<?, ?it/s]

0 크롤링
성공 : 1
url 주소 불일치 : 2
성공 : 3
성공 : 4
사이즈 정보 없음 : 5
할인가격 없음 : 6
데이터 수 불일치 : 6
성공 : 7
성공 : 8
성공 : 9
성공 : 10
사이즈 정보 없음 : 11
성공 : 12
성공 : 13
할인가격 없음 : 14
성공 : 14
할인가격 없음 : 15
성공 : 15
성공 : 16
성공 : 17
사이즈 정보 없음 : 18
성공 : 19
성공 : 20
성공 : 21
할인가격 없음 : 22
사이즈 정보 없음 : 22
사이즈 정보 없음 : 23
성공 : 24
성공 : 25
할인가격 없음 : 26
성공 : 26
성공 : 27
성공 : 28
성공 : 29
성공 : 30
성공 : 31
성공 : 32
할인가격 없음 : 33
성공 : 33
할인가격 없음 : 34
성공 : 34
할인가격 없음 : 35
사이즈 정보 없음 : 35
성공 : 36
성공 : 37
할인가격 없음 : 38
사이즈 정보 없음 : 38
성공 : 39
오류 발생 : 40


  9%|▉         | 1/11 [04:04<40:40, 244.09s/it]

1 크롤링
성공 : 1
url 주소 불일치 : 2
성공 : 3
할인가격 없음 : 4
데이터 수 불일치 : 4
성공 : 5
성공 : 6
성공 : 7
성공 : 8
성공 : 9
성공 : 10
사이즈 정보 없음 : 11
성공 : 12
성공 : 13
할인가격 없음 : 14
성공 : 14
성공 : 15
할인가격 없음 : 16
오류 발생 : 16
성공 : 17
사이즈 정보 없음 : 18
성공 : 19
성공 : 20
성공 : 21
할인가격 없음 : 22
성공 : 22
할인가격 없음 : 23
성공 : 23
성공 : 24
성공 : 25
성공 : 26
성공 : 27
할인가격 없음 : 28
성공 : 28
성공 : 29
성공 : 30
할인가격 없음 : 31
성공 : 31
할인가격 없음 : 32
성공 : 32
할인가격 없음 : 33
성공 : 33
성공 : 34
사이즈 정보 없음 : 35
성공 : 36
할인가격 없음 : 37
성공 : 37
성공 : 38
할인가격 없음 : 39
사이즈 정보 없음 : 39
할인가격 없음 : 40


 18%|█▊        | 2/11 [08:21<37:44, 251.64s/it]

...

 82%|████████▏ | 9/11 [32:30<06:46, 203.48s/it]

9 크롤링
성공 : 1
url 주소 불일치 : 2
성공 : 3
성공 : 4
성공 : 5
할인가격 없음 : 6
성공 : 6
데이터 수 불일치 : 7
성공 : 8
할인가격 없음 : 9
성공 : 9
성공 : 10
사이즈 정보 없음 : 11
사이즈 정보 없음 : 12
할인가격 없음 : 13
성공 : 13
할인가격 없음 : 14
성공 : 14
할인가격 없음 : 15
성공 : 15
성공 : 16
성공 : 17
성공 : 18
성공 : 19
할인가격 없음 : 20
성공 : 20
성공 : 21
성공 : 22
성공 : 23
할인가격 없음 : 24
성공 : 24
사이즈 정보 없음 : 25
오류 발생 : 26
오류 발생 : 27
오류 발생 : 28
오류 발생 : 29
오류 발생 : 30
오류 발생 : 31
오류 발생 : 32
오류 발생 : 33
오류 발생 : 34
오류 발생 : 35
오류 발생 : 36
오류 발생 : 37
오류 발생 : 38
오류 발생 : 39
오류 발생 : 40


 91%|█████████ | 10/11 [35:47<03:21, 201.37s/it]

10 크롤링
성공 : 1
url 주소 불일치 : 2
사이즈 정보 없음 : 3
데이터 수 불일치 : 4
사이즈 정보 없음 : 5
성공 : 6
할인가격 없음 : 7
성공 : 7
성공 : 8
할인가격 없음 : 9
성공 : 9
사이즈 정보 없음 : 10
할인가격 없음 : 11
성공 : 11
성공 : 12
성공 : 13
성공 : 14
할인가격 없음 : 15
데이터 수 불일치 : 15
성공 : 16
오류 발생 : 17
성공 : 18
할인가격 없음 : 19
성공 : 19
성공 : 20
사이즈 정보 없음 : 21
데이터 수 불일치 : 22
성공 : 23
성공 : 24
성공 : 25
오류 발생 : 26
오류 발생 : 27
오류 발생 : 28
오류 발생 : 29
오류 발생 : 30
오류 발생 : 31
오류 발생 : 32
오류 발생 : 33
오류 발생 : 34
오류 발생 : 35
오류 발생 : 36
오류 발생 : 37
오류 발생 : 38
오류 발생 : 39
오류 발생 : 40


100%|██████████| 11/11 [39:10<00:00, 213.70s/it]
